In [1]:
import json

# Read line-by-line JSON entries into a list
data = []
with open("train.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
import numpy as np
import json

# One-hot encoding for 4 POS tags
def one_hot(tag, num_tags=4):
    vec = np.zeros(num_tags)
    vec[tag - 1] = 1
    return vec

# Recurrent Perceptron with BPTT
class RecurrentPerceptron:
    def __init__(self, input_dim=4, hidden_dim=16, output_dim=1, lr=0.01):
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.W_in = np.random.randn(hidden_dim, input_dim) * 0.1
        self.W_h = np.random.randn(hidden_dim, hidden_dim) * 0.1
        self.b_h = np.zeros((hidden_dim, 1))

        self.W_out = np.random.randn(output_dim, hidden_dim) * 0.1
        self.b_out = np.zeros((output_dim, 1))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_deriv(self, y):
        return y * (1 - y)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_deriv(self, h):
        return 1 - h**2

    def forward(self, inputs):
        h = np.zeros((self.hidden_dim, 1))
        hs = []
        ys = []

        for x in inputs:
            x = x.reshape(-1, 1)
            h = self.tanh(self.W_in @ x + self.W_h @ h + self.b_h)
            y = self.sigmoid(self.W_out @ h + self.b_out)
            hs.append(h)
            ys.append(y)
        return ys, hs

    def bptt(self, input_tags, target_labels):
        inputs = [one_hot(tag) for tag in input_tags]
        targets = [np.array([[label]]) for label in target_labels]

        ys, hs = self.forward(inputs)

        dW_in = np.zeros_like(self.W_in)
        dW_h = np.zeros_like(self.W_h)
        db_h = np.zeros_like(self.b_h)
        dW_out = np.zeros_like(self.W_out)
        db_out = np.zeros_like(self.b_out)

        dh_next = np.zeros((self.hidden_dim, 1))

        for t in reversed(range(len(inputs))):
            dy = ys[t] - targets[t]
            dW_out += dy @ hs[t].T
            db_out += dy

            dh = self.W_out.T @ dy + dh_next
            dtanh = dh * self.tanh_deriv(hs[t])
            dW_in += dtanh @ inputs[t].reshape(1, -1)
            db_h += dtanh
            if t > 0:
                dW_h += dtanh @ hs[t - 1].T
                dh_next = self.W_h.T @ dtanh

        # Gradient descent
        for param, dparam in zip(
            [self.W_in, self.W_h, self.b_h, self.W_out, self.b_out],
            [dW_in, dW_h, db_h, dW_out, db_out]
        ):
            param -= self.lr * dparam

    def predict(self, pos_tags):
        inputs = [one_hot(tag) for tag in pos_tags]
        ys, _ = self.forward(inputs)
        return [int(y[0][0] > 0.5) for y in ys]

In [4]:
# Initialize and train
model = RecurrentPerceptron()

for epoch in range(30):
    for sample in data:
        model.bptt(sample["pos_tags"], sample["chunk_tags"])
    if epoch % 5 == 0:
        print(f"Epoch {epoch} complete")

Epoch 0 complete
Epoch 5 complete
Epoch 10 complete
Epoch 15 complete
Epoch 20 complete
Epoch 25 complete


In [5]:
# Read line-by-line JSON entries into a list
data1 = []
with open("test.jsonl", "r") as f:
    for line in f:
        data1.append(json.loads(line))

In [6]:
# Show predictions on training data
for sample in data1[:5]:  # show first 5 samples
    pred = model.predict(sample["pos_tags"])
    print(f"\nTokens:     {sample['tokens']}")
    print(f"POS Tags:   {sample['pos_tags']}")
    print(f"True Chunks:{sample['chunk_tags']}")
    print(f"Predicted:  {pred}")


Tokens:     ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
POS Tags:   [1, 4, 1, 4, 1, 1, 4, 1, 4, 2, 1, 4]
True Chunks:[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1]
Predicted:  [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1]

Tokens:     ['Nadim', 'Ladki']
POS Tags:   [1, 1]
True Chunks:[1, 0]
Predicted:  [1, 0]

Tokens:     ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06']
POS Tags:   [1, 4, 1, 1, 1, 4]
True Chunks:[1, 1, 1, 0, 0, 0]
Predicted:  [1, 0, 1, 0, 0, 1]

Tokens:     ['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1', 'win', 'against', 'Syria', 'in', 'a', 'Group', 'C', 'championship', 'match', 'on', 'Friday', '.']
POS Tags:   [1, 4, 2, 1, 4, 4, 3, 1, 1, 4, 2, 3, 4, 4, 4, 1, 4, 2, 1, 1, 1, 1, 4, 1, 4]
True Chunks:[1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
Predicted:  [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]

Tokens: 

In [7]:
def calculate_accuracy(model, data):
    correct_predictions = 0
    total_predictions = 0
    for sample in data:
        predicted_chunks = model.predict(sample["pos_tags"])
        true_chunks = sample["chunk_tags"]

        # Ensure the lengths are the same before comparison
        min_len = min(len(predicted_chunks), len(true_chunks))
        for i in range(min_len):
            if predicted_chunks[i] == true_chunks[i]:
                correct_predictions += 1
            total_predictions += 1

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

# Calculate accuracy on the training data
train_accuracy = calculate_accuracy(model, data)
print(f"\nTraining Accuracy: {train_accuracy:.4f}")

# Calculate accuracy on the test data
test_accuracy = calculate_accuracy(model, data1)
print(f"Test Accuracy: {test_accuracy:.4f}")



Training Accuracy: 0.7822
Test Accuracy: 0.7786
